In [1]:
!wget -O archive.zip https://www.dropbox.com/s/5yjv2hth42wbjz3/archive.zip?dl=0
!unzip "archive.zip"
!pip install emot

--2021-02-10 16:48:47--  https://www.dropbox.com/s/5yjv2hth42wbjz3/archive.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.18, 2620:100:6032:18::a27d:5212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/5yjv2hth42wbjz3/archive.zip [following]
--2021-02-10 16:48:47--  https://www.dropbox.com/s/raw/5yjv2hth42wbjz3/archive.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1cc18057b3632e53cc5b2b566c.dl.dropboxusercontent.com/cd/0/inline/BIq7ZN6iq27_sAf3uL83-3LQrYo2DLG1VAzQqPHQUghmLofR2WZF1gk0iQx2pOm88TITnuKYz-ln8uV8rYHoTs8n6JbsN5vMpmDzeNCCS6Z1LP06mqemvacvuS3OhctwZ2A/file# [following]
--2021-02-10 16:48:48--  https://uc1cc18057b3632e53cc5b2b566c.dl.dropboxusercontent.com/cd/0/inline/BIq7ZN6iq27_sAf3uL83-3LQrYo2DLG1VAzQqPHQUghmLofR2WZF1gk0iQx2pOm88TITnuKYz-ln8uV8rYHoTs8n6JbsN5vMpmDzeNCCS6

In [82]:
import pandas as pd
import numpy as np
import string 
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import re
from emot.emo_unicode import EMOTICONS
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
stopwords.words('english')
%matplotlib inline

def get_response(data, x):
    row = data[data.tweet_id == int(x)]
    return '' if (len(row) == 0) else row.iloc[0]['text']

df = pd.read_csv("twcs/twcs.csv")
df.sort_values(by='tweet_id', inplace=True)

apple_support = df[df.author_id == "AppleSupport"]
apple_support = apple_support[~apple_support.in_response_to_tweet_id.isna()]
apple_support = apple_support[['tweet_id', 'author_id', 'text', 'in_response_to_tweet_id']]
apple_support.loc[:, 'in_response_to_tweet_id'] = apple_support.in_response_to_tweet_id.apply(lambda x: get_response(df, x))
apple_support.rename(columns = {'author_id':'support_name', 'text':'answer', 'in_response_to_tweet_id':'question'}, inplace = True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
tweet_id  author_id     inbound  created_at                      text                                                                                                                                                response_tweet_id  in_response_to_tweet_id
2987918   ArgosHelpers  False    Thu Nov 30 07:58:42 +0000 2017  @823859 Hi Natalie that doesn't sound good. Can you follow us and DM us your order number so we can look at this for you ? - Will                 

In [ ]:
chat_words = {
    "afaik": "as far as i know",
    "asap": "as soon as possible",
    "atk": "at the keyboard",
    "atm": "at the moment",
    "brb": "be right back",
    "brt": "be right there",
    "btw": "by the way",
    "b4": "before",
    "cu": "see you",
    "cya": "see you",
    "faq": "frequently asked questions",
    "fc": "fingers crossed",
    "fyi": "for your information",
    "gn": "good night",
    "gr8": "great",
    "g9": "genius",
    "ic": "i see",
    "imo": "in my opinion",
    "iow": "in other words",
    "lol": "laughing out loud",
    "l8r": "later",
    "mte": "my thoughts rxactly",
    "m8": "mate",
    "nrn": "no reply necessary",
    "oic": "oh i see",
    "rofl": "rolling on the floor laughing",
    "thx": "thank you",
    "ttyl": "talk to you later",
    "u": "you",
    "u2": "you too",
    "w8": "wait",
    "imma": "i am going to",
    "2nite": "tonight",
    "dmed": "mesaged",
    'dm': "message"
}

contractions = {
    "ain't": "are not",
    "aren't": "are not",
    "'bout": "about",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "here's": "here is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "I would",
    "i'd've": "I would have",
    "i'll": "I will",
    "i'll've": "I will have",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "i phone": "iphone",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that had",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "'til": "until",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have",
}

def remove_chat_words_and_contractions(text):
    new_text = []
    for word in text.split(' '):
        if word in chat_words.keys():
            new_text += chat_words[word].split(' ')
        else:
            new_text.append(word)

        if word in contractions.keys():
            new_text += contractions[word].split(' ')
        else:
            new_text.append(word)
    return ' '.join(new_text)

def remove_urls(text):
    return re.sub(r'https://t\.co/\w+', '<url>', text)

def remove_mentions(text):
    return re.sub(r'@[0-9A-Za-z_\-]+', '<user>', text)

def remove_hashtags(text):
    return re.sub(r'#[0-9A-Za-z_\-]+', '', text)

def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

few_stopwords = ["a", "the", "an"]
nltk_stopwords = stopwords.words()

def remove_stopwords(text):
    return " ".join([word for word in nltk.word_tokenize(text) if not word in nltk_stopwords])

def lemmatize(text):
    return " ".join([lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in nltk.word_tokenize(text)])

def clean(text):
    text = re.sub(r'[“”]', '"', text)
    text = re.sub(r"’", "'", text)
    text = text.lower()
    text = remove_chat_words(text)
    text = remove_contractions(text)
    text = remove_urls(text)
    text = remove_mentions(text)
    text = remove_hashtags(text)
    text = re.sub(r'&gt;', '>', text)
    text = re.sub(r'&lt;', '<', text)
    text = remove_emojis(text)
    text = remove_emoticons(text)
    text = remove_stopwords(text)
    text = lemmatize(text)
    return text

apple_support.loc[:, 'question'] = apple_support.question.apply(lambda x: clean(x))
apple_support.loc[:, 'answer'] = apple_support.answer.apply(lambda x: clean(x))

I paid 800 for a phone for the battery to last me about four hours. @115858 your products are garbage
paid 800 phone battery last four hour . < user > product garbage

@435876 We know battery life is important and would like to help. DM us which iPhone and we'll start there. https://t.co/GDrqU22YpT
< user > know battery life important would like help . message u iphone start . < url >


@AppleSupport any clue as to when you will fix this I issue ?
< user > clue fix issue ?

@664246 We've got some info to share that can help. iOS 11.1.1 was recently released and it includes a fix for autocorrect issues. Be sure to back up your device prior to updating, and let us know if the issue persists afterwards. How to back up: https://t.co/4f8hwT5to6
< user > get info share help . io 11.1.1 recently release include fix autocorrect issue . sure back device prior update , let u know issue persists afterwards . back : < url >


@AppleSupport I hope you guys are working on an additional iOS 11 update